In [6]:
import os
import requests
import json
import datetime
import pandas as pd
import platform
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import re
import logging

# 로깅 설정
logging.basicConfig(level=logging.INFO)

def search_naver_shopping(query, display):
    client_id = os.getenv('NAVER_CLIENT_ID')  # 네이버 API 클라이언트 ID
    client_secret = os.getenv('NAVER_CLIENT_SECRET')  # 네이버 API 클라이언트 시크릿
    
    if not client_id or not client_secret:
        messagebox.showerror("API Error", '네이버 API 클라이언트 ID 또는 시크릿이 설정되지 않았습니다.')
        return []
    
    url = f'https://openapi.naver.com/v1/search/shop.json?query={query}&display={display}'
    headers = {
        'X-Naver-Client-Id': client_id,
        'X-Naver-Client-Secret': client_secret
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        data = response.json()
        if 'items' in data:
            return data['items']
        else:
            messagebox.showerror("Error", '응답에 "items" 키가 없습니다.')
            return []
    except requests.exceptions.HTTPError as http_err:
        messagebox.showerror("HTTP Error", f'HTTP 오류 발생: {http_err}')
    except requests.exceptions.ConnectionError as conn_err:
        messagebox.showerror("Connection Error", f'연결 오류 발생: {conn_err}')
    except requests.exceptions.Timeout as timeout_err:
        messagebox.showerror("Timeout Error", f'타임아웃 오류 발생: {timeout_err}')
    except requests.exceptions.RequestException as req_err:
        messagebox.showerror("Request Error", f'요청 오류 발생: {req_err}')
    except json.JSONDecodeError as json_err:
        messagebox.showerror("JSON Error", f'JSON 디코딩 오류 발생: {json_err}')
    except Exception as err:
        messagebox.showerror("Unknown Error", f'예상치 못한 오류 발생: {err}')

    return []

def sanitize_filename(filename):
    # 파일명에 사용할 수 없는 문자 제거
    return re.sub(r'[\\/*?:"<>|]', "", filename)

def save_results_to_file(items, filename):
    sanitized_filename = sanitize_filename(filename)
    today_date = datetime.datetime.now().strftime("%Y-%m-%d")
    full_filename = f"{sanitized_filename}_{today_date}.xlsx"
    
    try:
        for i, item in enumerate(items, start=1):
            item['순번'] = i
            item['lprice'] = int(item['lprice']) if item['lprice'] else 0
        
        df = pd.DataFrame(items)
        
        cols = ['순번'] + [col for col in df.columns if col != '순번' and col != 'hprice']  # '최고가'를 제외
        df = df[cols]
        
        df.to_excel(full_filename, index=False)
        
        messagebox.showinfo("Success", f"검색 결과가 파일 '{full_filename}'에 저장되었습니다.")
        
        if platform.system() == 'Windows':
            os.startfile(full_filename)
        elif platform.system() == 'Darwin':
            os.system(f'open "{full_filename}"')
        else:
            os.system(f'xdg-open "{full_filename}"')
    except IOError as io_err:
        messagebox.showerror("File Error", f'파일 쓰기 오류 발생: {io_err}')
    except Exception as err:
        messagebox.showerror("Unknown Error", f'예상치 못한 오류 발생: {err}')

def print_results(items):
    for item in items:
        tree.insert('', 'end', values=(
            item['순번'], item['title'], item['lprice'],
            item['mallName'], item['brand'], item['maker'],
            item['category1'], item['category2'], item['category3'], item['category4']
        ))

def search_and_save():
    query = query_entry.get()
    display = display_entry.get()
    filename = filename_entry.get()
    
    if not query or not display.isdigit() or not filename:
        messagebox.showwarning("Input Error", "모든 입력 값을 확인하세요.")
        return
    
    if int(display) <= 0:
        messagebox.showwarning("Input Error", "결과 개수는 양수이어야 합니다.")
        return
    
    tree.delete(*tree.get_children())  # 이전 검색 결과 삭제
    
    items = search_naver_shopping(query, int(display))
    if items:
        for i, item in enumerate(items, start=1):
            item['순번'] = i
        print_results(items)
        save_results_to_file(items, filename)

def on_exit():
    root.quit()
    root.destroy()

# GUI 설정
root = tk.Tk()
root.title("Naver Shopping 검색")
root.geometry("1050x600")

# 스타일 설정
style = ttk.Style()
style.theme_use("clam")

# 메뉴바 추가
menubar = tk.Menu(root)
file_menu = tk.Menu(menubar, tearoff=0)
file_menu.add_command(label="Exit", command=on_exit)
menubar.add_cascade(label="File", menu=file_menu)
root.config(menu=menubar)

# 메인 프레임
main_frame = ttk.Frame(root, padding="10")
main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# 입력 프레임
input_frame = ttk.LabelFrame(main_frame, text="검색 설정", padding="10")
input_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S), pady=10)

query_label = ttk.Label(input_frame, text="검색어:")
query_label.grid(row=0, column=0, sticky=tk.W, padx=5, pady=5)
query_entry = ttk.Entry(input_frame, width=40)
query_entry.grid(row=0, column=1, padx=5, pady=5)

display_label = ttk.Label(input_frame, text="결과 개수:")
display_label.grid(row=1, column=0, sticky=tk.W, padx=5, pady=5)
display_entry = ttk.Entry(input_frame, width=10)
display_entry.grid(row=1, column=1, padx=5, pady=5)

filename_label = ttk.Label(input_frame, text="파일명:")
filename_label.grid(row=2, column=0, sticky=tk.W, padx=5, pady=5)
filename_entry = ttk.Entry(input_frame, width=40)
filename_entry.grid(row=2, column=1, padx=5, pady=5)

# 버튼 프레임
button_frame = ttk.Frame(input_frame)
button_frame.grid(row=3, column=0, columnspan=2, pady=10)

search_button = ttk.Button(button_frame, text="검색 및 저장", command=search_and_save)
search_button.grid(row=0, column=0, padx=5)

# 종료 버튼 추가
exit_button = ttk.Button(button_frame, text="종료", command=on_exit)
exit_button.grid(row=0, column=1, padx=5)

# 결과 프레임
result_frame = ttk.LabelFrame(main_frame, text="검색 결과", padding="10")
result_frame.grid(row=1, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# Treeview 위젯
columns = ('순번', '상품명', '최저가', '쇼핑몰 이름', '브랜드명', '제조사', '카테고리1', '카테고리2', '카테고리3', '카테고리4')
tree = ttk.Treeview(result_frame, columns=columns, show='headings', height=15)

for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=100)

tree.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

# 스크롤바 추가
scrollbar = ttk.Scrollbar(result_frame, orient="vertical", command=tree.yview)
scrollbar.grid(row=0, column=1, sticky=(tk.N, tk.S))
tree.configure(yscrollcommand=scrollbar.set)

# 상태바 추가
statusbar = ttk.Label(root, text="상태: 준비", anchor=tk.W)
statusbar.grid(row=2, column=0, sticky=(tk.W, tk.E))

# 반응형 레이아웃 설정
main_frame.columnconfigure(0, weight=1)
main_frame.rowconfigure(1, weight=1)
result_frame.columnconfigure(0, weight=1)
result_frame.rowconfigure(0, weight=1)

root.mainloop()
